In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,214,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,10/01/2023,21/01/2023 12:00 HORAS,MADRE DE DIOS/TAHUAMANU/IBERIA,KM 597,(SECTOR IBERIA) PARO INDEFINIDO SOLICITANDO NU...,-11.4068843,...,-11.4068843+-69.4784365,2023 12:00 HORAS+01+21+12:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
3,225,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,21/01/2023 12:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 12:00 HORAS+01+21+12:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
4,227,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,08/01/2023,21/01/2023 12:00 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402,(SAN BERNARDO) PARO INDEFINIDO SOLICITANDO NUE...,-12.6898325,...,-12.6898325+-69.4042951,2023 12:00 HORAS+01+21+12:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,162,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,18/01/2023,21/01/2023 12:00 HORAS,HUANCAVELICA/HUANCAVELICA/HUANCAVELICA,KM 219,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.774253,...,-12.774253+-75.010555,2023 12:00 HORAS+01+21+12:00,0,0,UD. HUANCAVELICA,HUMANO,PE-26,LACHOJ - HUANCAVELICA SECTOR,SI,interrumpido
6,163,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,18/01/2023,21/01/2023 12:00 HORAS,HUANCAVELICA/HUANCAVELICA/YAULI,KM 230,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.777176,...,-12.777176+-74.931816,2023 12:00 HORAS+01+21+12:00,0,0,UD. HUANCAVELICA,HUMANO,PE-26,LACHOJ - HUANCAVELICA SECTOR,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,203,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,21/01/2023 12:00 HORAS,LA LIBERTAD/SANCHEZ CARRION/HUAMACHUCO,KM 1106,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-7.804518995,...,-7.804518995+-77.99389572,2023 12:00 HORAS+01+21+12:00,0,0,CONCESIONARIA CONVIAL SIERRA NORTE,HUMANO,PE-3N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
107,202,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,21/01/2023 12:00 HORAS,LA LIBERTAD/SANCHEZ CARRION/HUAMACHUCO,KM 1100,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-7.821478781,...,-7.821478781+-78.02545525,2023 12:00 HORAS+01+21+12:00,0,1,COMISARIA PNP HUAMACHUCO,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
108,200,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,21/01/2023 12:00 HORAS,LA LIBERTAD/SANCHEZ CARRION/HUAMACHUCO,KM 177+500,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-7.830480544,...,-7.830480544+-78.065214,2023 12:00 HORAS+01+21+12:00,0,0,COMISARIA PNP HUAMACHUCO,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
109,328,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,21/01/2023 12:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 391,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.9903469087213,...,-8.9903469087213+-75.452262878851,2023 12:00 HORAS+01+21+12:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
